# Exporting & Invoking Linux Kernel Symbols

When modules are loaded, they are dynamically linked into the kernel.As with user-space, dynamically linked binaries can call only into external functions explicitly exported for use. In the kernel, this is handled via special directives called `EXPORT_SYMBOL()` and `EXPORT_SYMBOL_GPL()`.  
Exported functions are available for use by modules. Functions not exported cannot be invoked from modules.The linking and invoking rules are much more stringent for modules than code in the core kernel image. Core code can call any nonstatic interface in the kernel because all core source files are linked into a single base image. Exported symbols, of course, must be nonstatic, too.

We're going to write a demo of `Hello World` level to learn:
- How to make a function available for others to use.
- How to use functions given by other modules.

## 1. Task
- Implement a module `hello_api.ko` to export symbols as APIs to be other modules.
- Implement an other module `hello_feature.ko` to invoke the APIs, the symbols exported from the previous module. 

## 2. Import a basic `hello world` demo

We are going to reuse the [`hello world`](../hello-kernel/linux_kernel_hello_world.ipynb) demo to build to new modules aforememtioned.

### 2.1 Makefile
This `Makefile` will build to the aforementioned modules.

In [1]:
%%writefile Makefile
obj-m              = hello_api.o
hello_api-objs     = hello_api_inexit.o

KERN_BUILD = /lib/modules/$(shell uname -r)/build
MODULE_DIR = $(PWD)

modules modules_install clean:
	make -C $(KERN_BUILD) M=$(MODULE_DIR) $@

Overwriting Makefile


### 2.2 The init and exit points for the module

In [2]:
%%writefile hello_api_inexit.c
/*
 * @desc: implement the init and exit point for hello_api.ko module.
*/
#include <linux/init.h>
#include <linux/module.h>
#include <linux/kernel.h>

MODULE_LICENSE("GPL");
MODULE_AUTHOR("Clymber Loong");
MODULE_DESCRIPTION("A hello world to export kernel symbols.");

static int hello_init(void)
{
    printk(KERN_INFO "[hello_api] Hello, World!\n");
    return 0;
}

static void hello_exit(void)
{
    printk(KERN_INFO "[hello_api] Bye bye, World!\n");
}

module_init(hello_init);
module_exit(hello_exit);

Overwriting hello_api_inexit.c


In [3]:
! make

make -C /lib/modules/6.1.28/build M=/home/clymber/Studio/kerdevel/export_symbol modules
make[1]: Entering directory '/home/clymber/Studio/@linux-6.1.28'
  CC [M]  /home/clymber/Studio/kerdevel/export_symbol/hello_api_inexit.o
  LD [M]  /home/clymber/Studio/kerdevel/export_symbol/hello_api.o
  MODPOST /home/clymber/Studio/kerdevel/export_symbol/Module.symvers
  CC [M]  /home/clymber/Studio/kerdevel/export_symbol/hello_api.mod.o
  LD [M]  /home/clymber/Studio/kerdevel/export_symbol/hello_api.ko
make[1]: Leaving directory '/home/clymber/Studio/@linux-6.1.28'


## 3. Implement an api and export it

At this point, we got a module that are almost identical with the `Hello World` module. Let's add an api to `hello_api` module.

In [4]:
%%writefile -a Makefile

hello_api-objs += hello_api_test.o

Appending to Makefile


In [5]:
%%writefile hello_api_test.c
/*
 * @desc: implement test APIs to be exported to the kernel. 
*/
#include <linux/kernel.h>

void api_test(void)
{
    printk(KERN_INFO "[hello_api] this is a test message.");
}
EXPORT_SYMBOL_GPL(api_test);

Overwriting hello_api_test.c


In [6]:
! make

make -C /lib/modules/6.1.28/build M=/home/clymber/Studio/kerdevel/export_symbol modules
make[1]: Entering directory '/home/clymber/Studio/@linux-6.1.28'
  CC [M]  /home/clymber/Studio/kerdevel/export_symbol/hello_api_test.o
  LD [M]  /home/clymber/Studio/kerdevel/export_symbol/hello_api.o
  MODPOST /home/clymber/Studio/kerdevel/export_symbol/Module.symvers
  CC [M]  /home/clymber/Studio/kerdevel/export_symbol/hello_api.mod.o
  LD [M]  /home/clymber/Studio/kerdevel/export_symbol/hello_api.ko
make[1]: Leaving directory '/home/clymber/Studio/@linux-6.1.28'


## 4. Use the api exported by module 'hello_api' from module 'hello_feature'

In [7]:
%%writefile hello_feature.c
/*
 * @desc: verify the symbol exported by module 'hello_api'
*/
#include <linux/init.h>
#include <linux/module.h>
#include <linux/kernel.h>

MODULE_LICENSE("GPL");
MODULE_AUTHOR("Clymber Loong");
MODULE_DESCRIPTION("A demo to invoke symbols that other modules exported.");

extern void api_test(void); // from hello_api.ko

static int hello_init(void)
{
    printk(KERN_INFO "[hello_feature] Hello, World!\n");
    api_test();
    return 0;
}

static void hello_exit(void)
{
    printk(KERN_INFO "[hello_feature] Bye Bye, World!\n");
}

module_init(hello_init);
module_exit(hello_exit);

Overwriting hello_feature.c


In [8]:
%%writefile -a Makefile

obj-m += hello_feature.o

Appending to Makefile


In [9]:
! make

make -C /lib/modules/6.1.28/build M=/home/clymber/Studio/kerdevel/export_symbol modules
make[1]: Entering directory '/home/clymber/Studio/@linux-6.1.28'
  CC [M]  /home/clymber/Studio/kerdevel/export_symbol/hello_feature.o
  MODPOST /home/clymber/Studio/kerdevel/export_symbol/Module.symvers
  CC [M]  /home/clymber/Studio/kerdevel/export_symbol/hello_feature.mod.o
  LD [M]  /home/clymber/Studio/kerdevel/export_symbol/hello_feature.ko
make[1]: Leaving directory '/home/clymber/Studio/@linux-6.1.28'


In [11]:
%%script bash
sudo dmesg -C
sudo insmod hello_api.ko && sudo insmod hello_feature.ko
sudo rmmod hello_feature && sudo rmmod hello_api
sudo dmesg -c

[ 8025.761890] [hello_api] Hello, World!
[ 8025.768803] [hello_feature] Hello, World!
[ 8025.768806] [hello_api] this is a test message.
[ 8025.775629] [hello_feature] Bye Bye, World!
[ 8025.798991] [hello_api] Bye bye, World!


In [12]:
! make clean

make -C /lib/modules/6.1.28/build M=/home/clymber/Studio/kerdevel/export_symbol clean
make[1]: Entering directory '/home/clymber/Studio/@linux-6.1.28'
  CLEAN   /home/clymber/Studio/kerdevel/export_symbol/Module.symvers
make[1]: Leaving directory '/home/clymber/Studio/@linux-6.1.28'
